In [ ]:
%pylab inline
import os
from matplotlib import pyplot as plt
import numpy as np
import logbook

In [ ]:
exp_name = "23_plot_path"
exp = logbook.Experiment(exp_name)
log = exp.get_logger()
path_dirs = [
    "archive/20_find_path_length/2017-05-05 13:28:24 dbe068d"
]
stats_file = "path_stats.csv"

In [ ]:
def get_mean_path_length(filename):
    all_nodes = set()
    total = 0
    with open(filename, "rb") as f:
        for row in f:
            project_id, source, target, length = row.rstrip().split(",")
            if source == target:
                continue
            total += int(length)
            all_nodes.add(int(target))
            all_nodes.add(int(source))
    node_count = len(all_nodes)
    pair_count = node_count * (node_count - 1)
    mean_length = total / float(pair_count)
    return mean_length

In [ ]:
with open(exp.get_filename(stats_file), "wb") as out:
    out.write("project_id,path_mean\n")
    for path_dir in path_dirs:
        for i, path_file in enumerate(os.listdir(path_dir)):
            if path_file.endswith('path_length.csv'):
                project_id = flow_file.split("-")[0]
                log.info("Loading path file: %s" % _file)
                path_length = get_mean_path_length(os.path.join(path_dir, path_file))
                out.write("%s,%d\n" % (project_id, path_length))
                out.flush()